In [23]:
import cv2
import hashlib
import numpy as np
from cryptography.fernet import Fernet
import base64
from datetime import datetime, timedelta



In [24]:
def derive_secure_key(user_key, pin):
    salt = hashlib.sha256(pin.encode()).digest()
    dk = hashlib.pbkdf2_hmac(
        'sha256',
        user_key.encode(),
        salt,
        100000
    )
    return base64.urlsafe_b64encode(dk)


In [25]:
def create_payload(secret_text, unlock_date):
    payload = f"{unlock_date.isoformat()}||{secret_text}"
    return payload


In [ ]:
def hide_text_with_timelock(
    image_path,
    secret_text,
    user_key,
    pin,
    output_path,
    months=None,
    unlock_at=None,
 ):
    """Hide encrypted text with a time lock.

    Provide either:
      - months=<int/float>: unlock time = now + 30*months days
      - unlock_at=<str|datetime>: explicit unlock date+time
        Supported string formats:
          - "YYYY-MM-DD HH:MM[:SS]"
          - "YYYY-MM-DDTHH:MM[:SS]"
    """
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Could not read image at path: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Determine unlock time
    if unlock_at is not None:
        if isinstance(unlock_at, str):
            try:
                unlock_date = datetime.fromisoformat(unlock_at)
            except ValueError as e:
                raise ValueError(
                    "Invalid unlock_at format. Use 'YYYY-MM-DD HH:MM[:SS]' or 'YYYY-MM-DDTHH:MM[:SS]'."
                ) from e
        elif isinstance(unlock_at, datetime):
            unlock_date = unlock_at
        else:
            raise TypeError("unlock_at must be a str or datetime")
    else:
        if months is None:
            raise ValueError("Provide either months or unlock_at")
        unlock_date = datetime.now() + timedelta(days=30 * float(months))

    payload = create_payload(secret_text, unlock_date)

    secure_key = derive_secure_key(user_key, pin)
    cipher = Fernet(secure_key)

    encrypted_data = cipher.encrypt(payload.encode())
    binary_data = ''.join(format(b, '08b') for b in encrypted_data)
    binary_data += '1111111111111110'  # delimiter (0xFF 0xFE)

    capacity_bits = img.shape[0] * img.shape[1] * 3
    if len(binary_data) > capacity_bits:
        raise ValueError(
            f"Message too large for this image. Need {len(binary_data)} bits, have {capacity_bits} bits."
        )

    idx = 0
    for row in img:
        for pixel in row:
            for i in range(3):
                if idx < len(binary_data):
                    pixel[i] = (pixel[i] & 0xFE) | int(binary_data[idx])
                    idx += 1

    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    print(f"🔒 Image locked until: {unlock_date}")

In [ ]:
def extract_text_with_timelock(stego_image, user_key, pin):
    img = cv2.imread(stego_image)
    if img is None:
        raise FileNotFoundError(f"Could not read image at path: {stego_image}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    secure_key = derive_secure_key(user_key, pin)
    cipher = Fernet(secure_key)

    binary_data = ""
    for row in img:
        for pixel in row:
            for i in range(3):
                binary_data += str(pixel[i] & 1)

    bytes_data = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        bytes_data.append(int(byte, 2))

    data = bytes(bytes_data)
    encrypted_data = data.split(b'\xff\xfe')[0]

    try:
        decrypted_payload = cipher.decrypt(encrypted_data).decode()
        unlock_time_str, message = decrypted_payload.split("||", 1)

        unlock_time = datetime.fromisoformat(unlock_time_str)
        current_time = datetime.now()

        if current_time < unlock_time:
            remaining = unlock_time - current_time
            total_seconds = int(remaining.total_seconds())
            days = total_seconds // 86400
            hours = (total_seconds % 86400) // 3600
            minutes = (total_seconds % 3600) // 60
            print("⏳ Access Denied!")
            print(f"Unlocks in: {days} days {hours} hours {minutes} minutes")
        else:
            print("🔓 Access Granted!")
            print("Message:")
            print(message)

    except:
        print("❌ Wrong KEY, PIN, or corrupted image")

In [32]:
hide_text_with_timelock(
    image_path="nature.png",
    secret_text="Open this only after 2 months",
    user_key="MyUltraSecretKey",
    pin="7788",
    months=0,
    output_path="time_locked_image.png"
)


🔒 Image locked until: 2026-01-18 12:01:04.390967


In [33]:
extract_text_with_timelock(
    stego_image="time_locked_image.png",
    user_key="MyUltraSecretKey",
    pin="7788"
)


🔓 Access Granted!
Message:
Open this only after 2 months
